Mini-projet : Interpolation de Données Manquantes
==============================

*Contexte* : dans le cadre d'acquisition de données sismiques, des capteurs (récepteurs) sont positionnés le long d'une ligne à la surface de la terre. Chacun enregistre au cours du temps le déplacement des particules (signal oscillant) qui varie lors du passage d'une onde acoustique ou élastique. Cet enregistrement s'appelle une **trace**. Chaque trace correspond à une *colonne* de l'image presentée au début de la section 2 (une seule position spatiale, un signal qui dépend du temps).

En pratique, il arrive que certains capteurs sont défaillants ou encore qu'ils ne sont pas présents dans certaines zones (pour des problèmes d'accessibilité par exemple). L'objectif principal du mini-projet est de reconstituer les traces manquantes.

*Hypothèses* : nous ferons l'hypothèse importante que les **signaux d'une trace peuvent être *localement* prédits à partir d'une trace voisine, par un simple décalage temporel** (voir la partie "Première Analyse" pour une définition précise). 

*Notions abordées* : 
* Analyse des signaux dans le domain d'origine ;
* Analyse des signaux après transformée de Fourier 2d ;
* Régularisation et formulation sous forme de problème inverse.

Pour cela, trois approches sont considerées :
* *Approche 1* : analyse dans le domaine $(x,t)$ : identification de 3 temps et de 3 pentes, sélection d'une ondelette source, prédiction des données manquantes ;
* *Approche 2* : transformée de Fourier 2d, identification des événements principaux, définition d'un masque et reconstitution des traces manquantes ;
* *Approche 3* (optionelle) : suite de l'approche 2 pour s'assurer que la prédiction respecte bien les observables. Elle se fait au travers de la résolution d'un problème inverse avec ajout de régularisation.

*Attentes* :
* Développement des approches 1 et 2 sur des données fournies ici ;
* Discusssion sur une comaparaison entre les approches 1 et 2, en particulier sur les avantages et limites des approches. Application de l'approche 1 ou 2 sur un exemple que vous pourrez créer vous-mêmes.

*Consignes* :
* **Très important** : pour toutes les figures, bien indiquer les axes avec les bonnes unités et des labels lisibles ;
* L'approche 3 est optionnelle.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Only for question 4 (optional)
from scipy.optimize import minimize

**Lecture des données**

In [ ]:
# Labelsize (display)
labelsize   = 14
    
# Load the input data
# Input data (with missing traces)
gpanel = np.load('gpanel.npy')
# Dense data (with all traces, only for comparison)
panel_dense = np.load('panel_dense.npy')
nt, nx = gpanel.shape
print("Dimension des panneaux (axes temps et espace):",nt,nx)

# Missing traces (index number)
mtr = [2,5,6,11,12,20,21,22,23]

# Vertical axis -- time
dt   = 3.125e-3  # increment (s)
at   = np.linspace(0,dt*(nt-1),nt)
# Horizontal axis -- space
dx   = 10. # increment (m)
ax   = np.linspace(0,dx*(nx-1),nx)

In [ ]:
first_sensor = gpanel[:,1]

fig = plt.figure()
av = plt.subplot(111)
plt.plot(at, first_sensor)
plt.title('Signal reçu par le premier capteur au cours du temps', fontsize = labelsize)
av.set_ylabel("Déplacement u (m)", fontsize = labelsize)
av.set_xlabel("Time (s)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.set_cmap('bwr')


**Affichage des données**

In [ ]:
# Display of the input data
vmax    = np.max(np.abs(gpanel))
fig = plt.figure()
av = plt.subplot(111)
plt.imshow(gpanel,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.title('Observables (avec traces manquantes, mises à zero)', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([-vmax,vmax])
plt.set_cmap('bwr')

fig = plt.figure()
av = plt.subplot(111)
plt.imshow(panel_dense,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.title('Observables non disponibles (seulement pour comparaison)', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([-vmax,vmax])
plt.set_cmap('bwr')

# Première Analyse
* Dans les 2 images plus haut, 3 événements principaux ressortent. Ils sont associés à 3 temps différents (pour $x = 0$ m) et à 3 pentes différentes représentées ici :

<img src="data.png" width="400">

* Nous ferons l'hypothèse que ces événements sont linéaires, c'est-à-dire qu'il est possible de prédire une trace à une position $x$ connaissant les temps $t_i$ à la position $x_0 = 0$ m, les pentes $p_i$ et une fonction $S(t)$ appelee ondelette source  :
$$u(t,x) \simeq \sum_{i=1}^{3} S(t - t_i - p_i(x-x_0))$$

* Quelle est l'unité des pentes $p_i$? Donner les valeurs approximatives de ces 3 pentes $p_i$ et des 3 temps $t_i$.

D'après la formule, les $p_i x$ sont homogènes à un temps, donc $[p_i] = T.L^{-1}$ soit la dimension de l'inverse d'une vitesse.

**Aliasing spatial**

Sur l'image suivante, des événements sont visibles à gauche, avec une pente bien visible $p_1$. Sur la partie de droite de l'image, deux pentes sont identifiables $p_2$ et $p_3$. Pourtant on peut suivre des événement de gauche à droite : c'est l'effet d'aliasing spatial.

<img src="aliasing.png" width="400">

Construisez des données comme suit, avec les mêmes échantillonages temporel et spatial que sur l'exemple avec les données manquantes : 
$$d(t,x) = \sin(2 \pi f_c (t - p\cdot x))$$
$f_c$ est la fréquence caractéristique du signal (pour l'obtenir, calculer les spectres de plusieurs traces et déterminer $f_c$ comme valeur pour laquelle le spectre est en moyenne le plus grand). Tracer $d(t,x)$ pour différentes valeurs de $p$, en commençant par $p=0$. La pente visible dans les données augmente avec $p$, jusqu'à un certain moment : c'est l'effet d'aliasing spatial. Est-il possible de retrouver le cas $p=0$ pour un $p$ non nul? A partir des observations, proposer un critère à respecter pour éviter l'aliasing spatial. Ce critère est une relation entre $p$, $f_c$ et la distance $dx$ (ici 10 m) entre 2 traces consécutives. Vérifier si les pentes $p_i$ conduisent ou pas à de l'aliasing spatial.

<font color = "#ee58f5">

## Détermination de $f_c$

In [ ]:
#Déterminer t1, t2 et t3.

plt.plot(gpanel[:,15])
plt.show()
 
t3 = at[60]
t1 = at[127]
t2 = at[350]

#p1 = (trace())/(ax[150]-ax[0])

print(at[127])


In [ ]:
def trace(x):
    return gpanel[:,x]

In [ ]:
plt.plot(trace(15))
plt.show()

t_p3 = 350
t_p1 = 225
t_p2 = 70

p1 = (at[t_p1] - t1) / 150
p2 = (at[t_p2] - t2) / 150
p3 = (at[t_p3] - t3) / 150

print(p1,p2,p3)

In [ ]:
def fc():
    #nt, nx = gpanel.shape
    sum_spectre = np.fft.fft(trace(0))
    for x in range(1,nx):
        sum_spectre += np.fft.fft(trace(x))
    return sum_spectre

In [ ]:
spectre = fc()[0:200]
plt.plot(spectre)
max = np.argmax(spectre[0:50])
print(max)

<font color = "#ee58f5">

La méthode utilisée pour trouver $f_c$ n'étant pas précisée, nous en appliquerons deux:
* pour chaque "trace", trouver la fréquence d'amplitude maximale puis faire la moyenne de ces fréquences
* effectuer la somme des spectres des traces puis trouver la fréquence d'amplitude maximale

In [ ]:
# choix de fréquences
frequences = np.fft.fftfreq(nt,dt)[:200]

# liste des fréquences maximales
fc_list = []
# somme des spectre
spectre_sum = np.zeros(len(frequences)) 
fig, ax1 = plt.subplots(figsize = (10,5))
for i in range(nx):
    if i not in mtr:
        fft = np.fft.fft(trace(i))[:200]
        spectre = 2.0/nt * np.abs(fft)
        
        spectre_sum += spectre # mise à jour de la somme
        fc_list.append(frequences[np.argmax(spectre)]) # stockage de la fréquence 
        
        ax1.plot(frequences, spectre)
ax1.set_xlabel("Fréquences [Hz]")
ax1.set_ylabel("Fourier spectre")
ax1.grid()

fc1 = np.mean(fc_list)
fc2 = frequences[np.argmax(spectre_sum)]
ax1.set_title(f"Spectres de toutes les traces : fc1 = {'%.2e' % fc1} Hz, fc2 = {'%.2e' % fc2} Hz")

print(frequences[199])

In [ ]:
def d(p, fc):
    d_p = np.zeros((nt,nx))
    for x in range(nx):
        for t in range(nt):
            d_p[t,x] = np.sin(2 * np.pi * fc * (t * dt - p * x * dx))
    return d_p

In [ ]:
l = [0, 1/4, 1/2, 3/4, 1]
n = len(l)
fc = fc1

plt.figure(figsize=(20,8))
plt.gcf().subplots_adjust(left = 0.125, bottom = 0.2, 
                                        right = 1.5, 
                                        top = 0.9, 
                                        wspace = 0.2, 
                                        hspace = 0)

i = 1
for k in l:
    
    plt.subplot(1,n,i)
    p_k = k /(fc * dx)
    d_p = d(p_k, fc)

    vmax = np.max(np.abs(d_p))
    plt.imshow(d_p,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
    plt.title(r"$d(x,t)$" + f"pour p = {p_k:.3f}", fontsize = labelsize)
    av.set_ylabel("Time (s)", fontsize = labelsize)
    av.set_xlabel("Position x (m)", fontsize = labelsize)
    av.tick_params(axis='both', which='major', labelsize=labelsize)
    plt.clim([-vmax,vmax])
    plt.set_cmap('bwr')

    i+=1

plt.show()

## Critère de non-repliment de spectre (alising)

# Approche 1 : Reconstitution des Données dans le Domaine $(x,t)$

## Détermination de $S(t)$

Sélectionner une trace (par exemple pour $x$ autour de 250 m et déterminer $S(t)$ avec l'hypothèse que $S(t)$ est maximale pour $t=0$. Prendre pour le support de $S$ une fenêtre de longueur autour de 0.2 s. Afficher le résultat.

In [ ]:
plt.plot(at-0.9,trace(25))
plt.plot(at[0:50],1.25*np.sinc(21*at[0:50]))
plt.plot()
plt.show()

In [ ]:
def S(t):
    if abs(t) < 0.19:
        return 1.25*np.sinc(21*t)
    else:
        return 0

In [ ]:
S_liste = [S(t) for t in np.linspace(-1,1,1000)]
plt.plot(np.linspace(-1,1,1000),S_liste)

## Détermination des $(t_i,p_i)$

A partir de la première trace en $x_0=0$, calculer pour plusieurs temps et pour plusieurs pentes les sommations suivantes :

$$F(t,p) = \Bigg[\int \,u(t-p\cdot(x-x_0),x)\,dx\Bigg]^2$$

A quoi correspondent les maxima de $F$? En déduire les valeurs de $(t_i,p_i)$ pour $i\in[1,3]$. Bien choisir l'échantillonnage en temps et en pente pour une bonne détermination des $(t_i,p_i)$. 

Remarques :
* Par exemple, il peut être intéressant d'interpoler linéairement entre 2 points voisins si $t - p\cdot(x-x_0)$ ne tombe pas sur un point de grille ;
* Pour une bonne estimation, l'échantillonnage des pentes doit être suffisament fin ;
* Il est possible de reprendre une idée du TP "synthèse musicale" pour sélectionner itérativement les meilleurs $(t_i,p_i)$, mais ici en 2d.

Reconstruire les données à partir de l'équation (1) et des valeurs $(t_i,p_i)$ sélectionnées. Commentez les résultats.

<font color = "#ee58f5">

## Réponse

Les maximas de la $F$ correspondent aux extremas de la fonction $(t,p) \mapsto \displaystyle \int_{x_0}^{x_m} \,u\big(t-p\cdot(x-x_0),x\big)\,dx$ où $x_m = 300m$. <br>
Cette fonction quantifie le déplacement algébrique total le long de la portion de droite comprise entre $x_0$ et $x_m$ d'équation $$y = t - p\cdot(x - x_0)$$ dans le plan $(x, y)$. <br>
Cette fonction présente un extrema pour les couples $(t, p)$ qui paramètrent les "lignes" (bleues et rouges) visibles sur l'image.

In [ ]:
def u_inter(t,x):
    alpha = x/dx
    beta = t/dt
    nx = int(x/dx)
    nt = int(t/dt)
    a = alpha - nx
    b = beta - nt
    try:
        res = 0.5 * ((1-a)*gpanel[nt,nx] + a*gpanel[nt+1,nx] + (1-b)*gpanel[nt,nx] + b*gpanel[nt,nx+1]) 
    except IndexError:
        return 0
    return res    

In [ ]:
def u(t,x):
    nx = int(x/dx)
    nt = int(t/dt)
    try:
        res = gpanel[nt,nx]
    except IndexError:
        return 0
    return res

In [ ]:
trace0_inter = [u_inter(t,0) for t in at]
trace0 = [u(t,0) for t in at]
plt.plot(trace0)
plt.plot(trace0_inter)

In [ ]:
def F(params):
    t,p = params 
    return -1 * np.sum([u_inter(t-p*x,x)**2 for x in ax])

In [ ]:
print(F((at[t_p1],p1)))
print(F((at[t_p2],p2)))
print(F((at[t_p3],p3)))

In [ ]:
minimize(F,(at[t_p1],p1),bounds = ((0,1.2),(-0.01,0.01)))

In [ ]:
minimize(F,(at[t_p2],p2),bounds = ((0,1.2),(-0.01,0.01)))

In [ ]:
minimize(F,(at[t_p3],p3), bounds = ((0,1.2),(-0.01,0.01)))

In [ ]:
minimize(F,(0,0), bounds = ((0,1.2),(-0.01,0.01)))

In [ ]:
# Annulation d'une droite
# Supposons avoir trouvé une droite pertinente. et donc un couple (t,p)

def cancel(ti,pi,eps):
    data = gpanel.copy()
    for x in ax:
        for t in at:
            if abs(t-(ti - pi*x)) < eps:
                data[int(t/dt),int(x/dx)] = 0
    vmax = np.max(np.abs(data))
    plt.imshow(data,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
    plt.title(r"Annulation de la droite $({pi},{ti})$", fontsize = labelsize)  
    av.set_ylabel("Time (s)", fontsize = labelsize)
    av.set_xlabel("Position x (m)", fontsize = labelsize)
    av.tick_params(axis='both', which='major', labelsize=labelsize)
    plt.clim([-vmax,vmax])
    plt.set_cmap('bwr')
    plt.show()

cancel(1.0937,0.006,0.1)

In [ ]:
cancel(0.21875,-0.00583333,0.1)

# Approche 2 : Analyse des Données dans le Domaine de Fourier 2d

## Approche théorique
Calculer la transformée de Fourier 2d de $u$ à partir de la transformée de Fourier 1d de $S$. Expliquer toujours à partir des équations pourquoi on s'attend à avoir 3 événements linéaires dans le domaine de Fourier. Pourquoi tous ces événements passent-ils par l'origine $(0,0)$ dans le domaine de Fourier? Donner une expression quantitative entre les pentes dans le domaine de Fourier et les pentes $p_i$.  

<font color = "#ee58f5">

On a choisi comme fonction d'ondelette la fonction 

$$
S :
\left\lbrace
\begin{array}{ccc}
\mathbb{R} & \longrightarrow & \mathbb{R} \\
t & \longmapsto & \mathbb{1}_{[-\tau, \tau]}sinc(\alpha t) 
\end{array}\right.
$$

où $\tau = 0.2s$ et $\alpha = ?$.

$S$ est de carré intégrable sur $\mathbb{R}$, ce qui assure que $u$ le soit sur $\mathbb{R}^2$ (en prolongeant à $L^{2}(\mathbb{R}^2)$ la transformée de Fourier sur l'espace de Schwartz). <br>
Par ailleurs , comme chacune des fonctions $S_i : (t, x) \in \mathbb{R}^2 \rightarrow S(t - t_i - p_i(x - x_0))$ est de carré intégrable, par linéarité de la transformée de Fourrier, on a :

$$
\begin{aligned}
\forall (\omega, \xi) \in \mathbb{R}^2 \ \hat{u}(\omega, \xi) &= \sum_{i = 1}^3 \hat{S_i}(\omega, \xi)\\
&= \sum_{i = 1}^3 \int_{\mathbb{R}} \int_{\mathbb{R}} S\big(t - t_i - p_i(x - x_0)\big) e^{-i(\xi x + \omega t)} \mathrm{d}t \: \mathrm{d}x 
\\
&= \sum_{i = 1}^3 \int_{\mathbb{R}} \int_{\mathbb{R}} S(v) e^{-i\omega v} \: e^{-i \Big(\xi x + \omega \big(t_i + p_i(x - x_0)\big)\Big)}\mathrm{d}t \: \mathrm{d}x 
\\
&= \sum_{i = 1}^3 \int_{\mathbb{R}} \hat{S}(\omega) \: e^{-i \Big(\xi x + \omega \big(t_i + p_i(x - x_0)\big)\Big)} \: \mathrm{d}x 
\\
&= \hat{S}(\omega)  \sum_{i = 1}^3 e^{-i \omega (t_i - p_i x_0)} \: \int_{\mathbb{R}} e^{-i (\xi + \omega p_i)x} \mathrm{d}x 
\\
&= \hat{S}(\omega) \sum_{i = 1}^3 e^{-i \omega (t_i - p_i x_0)} \: \delta(\xi + \omega p_i)
\end{aligned}
$$

Les écritures ci-dessus sont à prendre au sens de limites d'intégrales de fonctions dans l'espace de Schwartz. On se place dans le cadre de la théorie des distributions en introduisant la fonction de Dirac "$\delta$".

<font color = "#ee58f5">

On observe que comme son analogue dans le domaine spatial et temporel, la transformée de Fourier du déplacement de trois contributions. Ces contributions sont les termes en 
$\hat{S}(\omega) \: e^{-i \omega (t_i - p_i x_0)} \: \delta(\xi + \omega p_i)$ pour $i \in. \{1, 2, 3\}$. 
À $\omega$ et $\xi$ fixés, elles sont toutes nulles si et seulement si $\omega  = - \frac{1}{p_i} \xi$. Ces équations représentent des évènement linéaires de coeffficients 
directeurs $- \frac{1}{p_1}$, $- \frac{1}{p_2}$ et $- \frac{1}{p_3}$ dans le domaine de Fourier. <br>
Par ailleurs ces relations imposent que ces évènements passent tous par l'origine $(0, 0)$ du domaine de Fourier.

## Implémentation
Représenter la transformée de Fourier 2d et bien préciser les axes. Indiquer à quoi correspondent la fréquence maximale et le nombre d'onde maximal. Faire le lien entre les 3 événements dans l'espace $(t,x)$ et ceux dans l'espace de Fourier. Expliquer ce qu'il se passe aux bords dans le domaine de Fourier. N'hesitez pas à zoomer autour de la position centrale dans le domaine de Fourier.

Remarques : 
* Le nombre d'onde (en 1/m) est l'équivalent de la fréquence pour l'axe spatial ;
* Le signal a très peu d'énergie pour les fréquences au-delà de 20 Hz.

<font color = "#ee58f5">

# Réponse

Pour représenter la transformée de Fourier 2d, nous avons fait le choix de la repésenter sur une fenêtre centrée au tour de 
l'origine en utilisant donc `np.fft.fftshift`.

In [ ]:
fft2D_gpanel = np.abs(np.fft.fft2(gpanel))

freq_t = np.fft.fftshift(np.fft.fftfreq(nt, dt))
freq_x = np.fft.fftshift(np.fft.fftfreq(nx, dx))

data_fft2 = np.fft.fftshift(fft2D_gpanel)[:][160:240]

vmax = np.max(data_fft2)
axisx = freq_x
axisy = freq_t[160:240]
fig = plt.figure()
X, Y = np.meshgrid(axisx, axisy)
plt.pcolor(X, Y, data_fft2, shading='nearest')
plt.title("Transformée de Fourier du signal à l'aide de numpy", fontsize = labelsize)
plt.xlabel("Longueur d'onde ($m^{-1}$)", fontsize = labelsize)
plt.ylabel("Fréquence ($s^{-1}$)", fontsize = labelsize)
plt.set_cmap('viridis')
plt.clim(0,vmax)
plt.colorbar()

plt.text(-0.01, 7, r'$\alpha$', fontdict=None,
         backgroundcolor = 'white', 
         fontweight = 'bold')

plt.text(-0.0375, 1, r'$\beta$', fontdict=None,
         backgroundcolor = 'white', 
         fontweight = 'bold')

plt.text(-0.0375, 13, r'$\gamma$', fontdict=None,
         backgroundcolor = 'white', 
         fontweight = 'bold')


In [ ]:
import cmath

tau = 0.2
alpha = 21

t3 = at[60]
t1 = at[127]
t2 = at[350]

p1 = (at[225] - t1)/150
p2 = (at[65] - t2)/150
p3 = (at[350] - t3)/150

def f(w):
    S = np.sin((alpha - w) * tau) / (alpha - w)
    S += np.sin((alpha + w) * tau) / (alpha + w)  
    sum = 0
    for (ti, pi) in [(t1, p1), (t2, p2), (t3, p3)]:
        arg = - w * ti
        sum += complex(np.cos(arg), np.sin(arg))
    
    return np.abs(S * sum)


X = np.linspace(0, 20, 100)
Y1 = [(np.sin((alpha - x) * tau) / (alpha - x)) + (np.sin((alpha + x) * tau) / (alpha + x)) for x in X] 
Y2 = [f(x) for x in X]

plt.plot(X, Y1)
plt.plot(X, Y2)
plt.show()
#print(p1, p2, p3)

<font color = "#ee58f5">

La fréquence maximale et le nombre d'onde maximal sont donnés respectivement par 
$\frac{1}{\mathrm{d}t}$ et $\frac{1}{\mathrm{d}x}$. On remarque que les constantes $\mathrm{d}x$ et $\mathrm{d}t$ 
qui définissent les "fréquences d'échantillonage" limitent également la résolution.
    
On iditenfie les évènements dans le domaine de Fourier à ceux du domaine réel par leur pente, ainsi on a :
* $\alpha = -\frac{1}{p_1}$
* $\beta = -\frac{1}{p_2}$
* $\gamma = -\frac{1}{p_3}$

Le tracé du module de la transformée de Fourier est incohérent avec le graphique.

## Définition d'un masque dans le domaine de Fourier 2d
On note $f$ la fréquence et $k_x$ le nombre d'onde. Dans le domaine de Fourier 2d, on souhaite définir un masque avec des 1 qui couvrent les événements principaux. A partir de $(f,k_x)=(0,0)$, faire des sommations selon différentes pentes (dans le domaine de Fourier, après avoir pris le module des valeurs complexes) et tracer la valeur de ces sommes en fonction de la pente. On s'attend à ce que 3 valeurs ressortent. Calculer une fonction qui sélectionne automatiquement ces valeurs et construire un masque dans le domaine de Fourier 2d qui vaut 1 autour de ces trois valeurs principales et 0 en dehors.

Comment faire pour prendre en compte ce qui se passe aux bords dans le dommaine de Fourier?

In [ ]:
nt, nx = fft2D_gpanel.shape

In [ ]:
nf, nk = fft2D_gpanel.shape
# Vertical axis -- time
dt   = 3.125e-3  # increment (s)
df = 1/dt
af   = np.linspace(0,df*(nf-1),nf)
# Horizontal axis -- space
dx   = 10. # increment (m)
dk = 1/dx
ak   = np.linspace(0,dk*(nk-1),nk)

In [ ]:
def u(f,k):
    nk = int(k/dk)
    nf = int(f/df)
    try:
        res = fft2D_gpanel[nf,nk]
    except IndexError:
        return 0
    return res

def F(p):
    return -1 * np.sum([u(p*k,k) for k in ak])

F_liste = [F(p) for p in np.linspace()]
plt.plot(F())

## Application du masque et analyse des résultats
Appliquer le masque (sur les données après transformée de Fourier 2d, sans prendre le module des valeurs), puis appliquer la transformée de Fourier inverse. Commenter les résultats (comparaison à la fois dans les domaines $(t,x)$ et $(f,k_x)$. Si nécessaire, revenir à la question précédente pour changer le masque et avoir une meilleure interpolation des données. Expliquer ces changements. Discuter de l'importance du masque.

# Approche 3 : Amélioration de l'Approche 2 (*partie optionelle*)
Le résultat de l'approche 2 montre en pratique que les données prédites après transformée de Fourier inverse 2d ne calent pas parfaitement aux données observées (pour les traces non nulles). C'est un inconvénient, mais d'un autre côté, ceci offre la possibilité d'aller plus loin pour forcer un bon calage.

Pour cela, nous définissons une fonction objective

$$
J[p] = \frac{1}{2}||T p - p^\mathrm{obs}||_2^2 + \frac{\alpha}{2} || F p ||_M^2
$$

ou $p(t,x)$ sont les signaux (sur toutes les traces), $T$ l'operateur qui vaut 1 pour des traces existantes et 0 sinon, $p^\mathrm{obs}(t,x)$ les observables (i.e. pour les traces non nulles), $F$ la transformee de Fourier 2d. La norme $||p||_2^2 = \sum_x \sum_t |p(t,x)|^2$ et $||q||_M^2= \sum_x \sum_t M(f,k_x)|q(f,k_x)|^2$, avec $M$ un masque défini dans le domaine de Fourier. Le poids $\alpha$ est une pondération entre les deux termes.

On cherche à minimiser $J$. Le meilleur $p$ sera celui qui minimise l'écart aux données (premier terme) tout en minimisant l'énergie dans le domaine de Fourier 2d (avec présence du masque). Par rapport au masque $M_0$ des questions 2 et 3, on prend ici $M=1 - M_0$. Expliquer pourquoi.

La minimisation se fait de manière itérative, avec la fonction ```minimize``` de scipy (```from scipy.optimize import minimize```). Parmi plusieurs possibilités, je suggère de prendre une approche de gradient conjugué (CG). La minisation va se faire selon :

```
res = minimize(defj, p0, method='CG', jac=defg, \
        options={'disp': True, 'maxiter':niter})
```

Il faut donc créer une valeur initiale pour $p$ (ici ```p0```), spécifier le nombre d'itérations ```niter``` et surtour définir la fonction objective ```defj()``` et son gradient par rapport à $p$ ```defg()```.

Le gradient est donné par 

$$
\frac{\partial J}{\partial p} = T^t(T p - p^\mathrm{obs}) + \alpha F^t M F p
$$

où $^t$ est l'opérateur transposé. L'application de $T^t$ correspond donc à insérer des traces nulles là où il n'y a pas d'observables et $F^t$ est la transformée de Fourier inverse.

Construire les fonctions ```defj()``` et ```defg()```. Attention, ```minimize``` attend que ```p``` soit sous forme de vecteur. On peut donc utiliser ```np.transpose(p,nt*nx)``` (voir plus bas dans l'exemple).

$\alpha$ est une pondération dont la valeur est à tester.

La valeur initiale ```p0``` de $p$ peut être prise égale à ```gpanel``` (le tableau lu au debut du code).

Faire plusieurs itérations. Analyser les résultats à la fois dans le domaine temporel et dans le domaine de Fourier 2d. Etudier l'importance de la régulariation.

In [ ]:
# Exemple de structure du code
def defj(p):
    """Definition of the objective function"""
    # Retrieve the size
    pan = np.reshape(p,[nt,nx])
    j0  = 0.
    #
    # ADD
    #
    return j0
    
    
def defg(p):
    """Definition of the gradient of the objective function"""
    # Retrieve the size
    pan = np.reshape(p,[nt,nx])
    grd = np.zeros([nt,nx])
    #
    # ADD
    #
    # Return with vector size
    return np.reshape(grd,nt*nx)

In [ ]:
# Iterative mininisation (example)
niter = 10 # number of iterations
p0 = np.reshape(np.copy(gpanel),nt*nx)
res = minimize(defj, p0, method='cg', \
                jac=defg, \
                options={'disp': True, 'maxiter':niter})
# Final panel
panelf = np.reshape(res.x,[nt,nx])

# Analyse des Limites des Approches
Proposer une analyse des avantages et limites des approches 1, 2 (et 3 le cas échéant). Tester la robustesse au travers de d'autres applications. Par exemple, vous pouvez :
* Ajouter du bruit (gaussien ou non) sur les données ;
* Avoir des événements linéaires avec des amplitudes différentes ;
* Ajouter des délais en temps pour chaque position en $x$ et comparer les résultats ;
* Rnlever encore plus de données en entrée (plus de traces blanches) ;
* Application sur des images totalement différentes.

**Cette dernière partie est très ouverte, et je fais appel à votre créativité.**